In [1]:
import pandas as pd
import seaborn as sns
sns.set_theme(style="ticks", palette="pastel")
import plotly.graph_objects as go

C:\Users\khngu\AppData\Local\Temp\ipykernel_33528\3847934184.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df_transfers = pd.read_csv('../MIMIC-III-Visuals/data/mimic-iii-clinical-database-1.4/TRANSFERS.csv.gz')
df_transfers.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,EVENTTYPE,PREV_CAREUNIT,CURR_CAREUNIT,PREV_WARDID,CURR_WARDID,INTIME,OUTTIME,LOS
0,657,111,192123,254245.0,carevue,transfer,CCU,MICU,7.0,23.0,2142-04-29 15:27:11,2142-05-04 20:38:33,125.19
1,658,111,192123,NaN,carevue,transfer,MICU,NaN,23.0,45.0,2142-05-04 20:38:33,2142-05-05 11:46:32,15.13
2,659,111,192123,NaN,carevue,discharge,NaN,NaN,45.0,NaN,2142-05-05 11:46:32,NaN,NaN
3,660,111,155897,249202.0,metavision,admit,NaN,MICU,NaN,52.0,2144-07-01 04:13:59,2144-07-01 05:19:39,1.09
4,661,111,155897,NaN,metavision,transfer,MICU,NaN,52.0,32.0,2144-07-01 05:19:39,2144-07-01 06:28:29,1.15


In [3]:
# Drop rows where 'CURR_CAREUNIT' or 'PREV_CAREUNIT' is empty (meaning it's an admission or discharge)
df_sankey = df_transfers[(df_transfers['PREV_CAREUNIT'] != '') & (df_transfers['CURR_CAREUNIT'] != '')]

# Aggregate the transfers between care units
df_flow = df_sankey.groupby(['PREV_CAREUNIT', 'CURR_CAREUNIT']).size().reset_index(name='count')

# Create a list of unique care units
care_units = pd.concat([df_flow['PREV_CAREUNIT'], df_flow['CURR_CAREUNIT']]).unique()
care_unit_dict = {unit: i for i, unit in enumerate(care_units)}

# Map the care units to a source-target index
df_flow['source'] = df_flow['PREV_CAREUNIT'].map(care_unit_dict)
df_flow['target'] = df_flow['CURR_CAREUNIT'].map(care_unit_dict)

fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=care_units
    ),
    link=dict(
        source=df_flow['source'],
        target=df_flow['target'],
        value=df_flow['count']
    ))])

fig.update_layout(title_text="Patient Flow Through Hospital Departments", font_size=10)
fig.show()
